In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("241211_03_SparkSQL_UDF").getOrCreate()

24/12/11 16:08:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
datas = [
    ("A", "2022-04-16", 31200),
    ("B", "2022-04-17", 41200),
    ("C", "2022-04-11", 31500),
    ("D", "2022-04-12", 21500),
    ("E", "2022-04-13", 51000)
]
columns = ["product", "date", "price"]

In [3]:
df = spark.createDataFrame(data=datas, schema=columns)
df.show()

+-------+----------+-----+
|product|      date|price|
+-------+----------+-----+
|      A|2022-04-16|31200|
|      B|2022-04-17|41200|
|      C|2022-04-11|31500|
|      D|2022-04-12|21500|
|      E|2022-04-13|51000|
+-------+----------+-----+



# udf 

user defined function : query 에서 사용하는 사용자정의함수
1. 파이썬의 함수로 정의한다.
2. spark.udf.register() 로 등록한다.
3. sql 문 안에서 func 처럼 사용한다.

In [4]:
from pyspark.sql.types import LongType

def squared(n):
    return n*n
    

In [7]:
squared(3)

9

In [8]:
spark.udf.register('squared', squared, LongType())

<function __main__.squared(n)>

In [5]:
df.createOrReplaceTempView('product')

In [6]:
spark.sql('select * from product').show()

+-------+----------+-----+
|product|      date|price|
+-------+----------+-----+
|      A|2022-04-16|31200|
|      B|2022-04-17|41200|
|      C|2022-04-11|31500|
|      D|2022-04-12|21500|
|      E|2022-04-13|51000|
+-------+----------+-----+



In [10]:
spark.sql('select price, squared(price) from product').show()

+-----+--------------+
|price|squared(price)|
+-----+--------------+
|31200|     973440000|
|41200|    1697440000|
|31500|     992250000|
|21500|     462250000|
|51000|    2601000000|
+-----+--------------+



In [16]:
def read_number( n ) :
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    result = []
    i=0
    while n>0:
        n,r = divmod(n,10)
        if r>0:
            result.append(nums[r-1]+units[i])
        i += 1
    return "".join(reversed(result))

In [17]:
read_number(123)

'일백이십삼'

In [18]:
spark.udf.register('read_number', read_number)

<function __main__.read_number(n)>

In [19]:
spark.sql('select price, read_number(price) from product').show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|31200|      삼만일천이백|
|41200|      사만일천이백|
|31500|      삼만일천오백|
|21500|      이만일천오백|
|51000|          오만일천|
+-----+------------------+

